# Solutions Engineering Workload Modeling

## Project Overview

This Jupyter notebook models Solutions Engineering workload based on revenue targets, conversion rates, staffing levels, and activity assumptions. It enables scenario modeling to understand capacity requirements and utilization across different business conditions.

### Key Capabilities
- **Revenue-Driven Pipeline Modeling**: Calculate required activities from quarterly revenue goals
- **Interactive Parameter Controls**: Adjust assumptions and see real-time impact
- **Capacity Analysis**: Understand SE team utilization and identify constraints
- **Account Management Modeling**: Factor in existing customer meeting requirements
- **Strategic Activity Planning**: Allocate time for non-pipeline activities
- **Professional Visualizations**: Executive-ready charts and dashboards

### Business Applications
- Staffing planning and capacity forecasting
- Revenue target feasibility analysis
- Resource allocation optimization
- Scenario planning for different business conditions

## Introduction

### Workload Modeling Approach

This model uses a **revenue-driven approach** to calculate SE workload requirements:

1. **Start with Revenue Goals**: Quarterly revenue targets drive required new logo counts
2. **Work Backwards Through Funnel**: Calculate required demos, evaluations, and meetings
3. **Factor in Activity Types**: Different evaluation types require different SE time investments
4. **Add Account Management**: Existing customer meetings and onboarding requirements
5. **Include Strategic Activities**: Time for customer zero, content creation, and enablement
6. **Calculate Utilization**: Compare total workload to available SE capacity

### Key Assumptions
- **40-hour work week** standard capacity per SE
- **Conversion rates** remain consistent across evaluation types
- **Account meeting frequencies** based on customer segment strategy
- **Strategic activities** distributed between ICs and Directors based on role allocation

## Library Imports and Configuration

In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive widgets for parameter controls
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configure matplotlib for inline display
%matplotlib inline

# Set seaborn style for professional appearance
sns.set_style("whitegrid")
sns.set_palette("husl")

# Configure pandas display options for better table formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 2)

# Configure matplotlib for better chart appearance
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11

print("✅ All libraries imported successfully")
print("✅ Configuration applied")
print("📊 Ready to begin workload modeling")

## Table of Contents

### 📋 Core Sections
1. [**Helper Functions Framework**](#helper-functions) - Core calculation functions
2. [**Parameter Management**](#parameter-management) - Input parameter definitions and validation
3. [**Revenue & Pipeline Calculations**](#revenue-pipeline) - Core funnel math and workload calculations
4. [**Account Management Logic**](#account-management) - Meeting frequency and time allocation
5. [**Strategic Activities Logic**](#strategic-activities) - Strategic time allocation and distribution

### 🎛️ Interactive Interface
6. [**Interactive Parameter Controls**](#interactive-controls) - Widget-based parameter input
7. [**Real-time Calculations**](#real-time-calculations) - Live calculation updates

### 📊 Visualization & Results
8. [**Core Visualizations**](#core-visualizations) - Essential charts for utilization and capacity
9. [**Executive Dashboard**](#executive-dashboard) - Professional dashboard with multiple chart types
10. [**Results Summary & Export**](#results-summary) - Executive summary and export capabilities

### 🔧 Testing & Validation
11. [**Testing Framework**](#testing) - Comprehensive validation and testing
12. [**Usage Examples**](#examples) - Sample scenarios and use cases

---

# 1. Helper Functions Framework {#helper-functions}

*Core calculation functions that will be reused throughout the notebook*

In [ ]:
# Core helper functions for SE workload modeling calculations

def calculate_required_new_logos(quarterly_revenue_goal, average_selling_price):
    """
    Calculate the number of new logos needed to meet quarterly revenue goals.
    
    Parameters:
    - quarterly_revenue_goal (float): Target revenue for the quarter
    - average_selling_price (float): Average deal size
    
    Returns:
    - float: Number of new logos required
    """
    if average_selling_price <= 0:
        raise ValueError("Average selling price must be greater than 0")
    
    return quarterly_revenue_goal / average_selling_price


def calculate_weighted_win_rate(eval_mix_percentages, win_rates):
    """
    Calculate weighted average win rate across all evaluation types.
    
    Parameters:
    - eval_mix_percentages (dict): Percentages for each evaluation type
    - win_rates (dict): Win rates for each evaluation type
    
    Returns:
    - float: Weighted average win rate (0-1)
    """
    total_percentage = sum(eval_mix_percentages.values())
    if abs(total_percentage - 100) > 0.01:  # Allow small floating point errors
        raise ValueError(f"Evaluation mix percentages must sum to 100%, got {total_percentage}%")
    
    weighted_sum = 0
    for eval_type in eval_mix_percentages:
        if eval_type not in win_rates:
            raise ValueError(f"Win rate not provided for evaluation type: {eval_type}")
        weighted_sum += (eval_mix_percentages[eval_type] / 100) * (win_rates[eval_type] / 100)
    
    return weighted_sum


def calculate_required_activities(new_logos_needed, conversion_rates, eval_mix, win_rates):
    """
    Work backwards through conversion funnel to calculate required weekly activities.
    
    Parameters:
    - new_logos_needed (float): Required new logos per quarter
    - conversion_rates (dict): Conversion rates for each funnel stage
    - eval_mix (dict): Percentage mix of evaluation types
    - win_rates (dict): Win rates by evaluation type
    
    Returns:
    - dict: Required weekly activities by type
    """
    # Calculate weighted win rate
    weighted_win_rate = calculate_weighted_win_rate(eval_mix, win_rates)
    
    # Work backwards through funnel (quarterly to weekly)
    weeks_per_quarter = 13
    
    # Required evaluations per quarter
    required_evaluations_quarterly = new_logos_needed / weighted_win_rate
    
    # Required demos per quarter (from tech eval conversion rate)
    required_demos_quarterly = required_evaluations_quarterly / (conversion_rates['tech_eval_conversion_rate'] / 100)
    
    # Required opportunities per quarter (from demo conversion rate)
    required_opportunities_quarterly = required_demos_quarterly / (conversion_rates['demo_conversion_rate'] / 100)
    
    # Required initial meetings per quarter (from opportunity creation rate)
    required_meetings_quarterly = required_opportunities_quarterly / (conversion_rates['opportunity_creation_rate'] / 100)
    
    # Convert to weekly and break down evaluations by type
    weekly_activities = {
        'initial_meetings': required_meetings_quarterly / weeks_per_quarter,
        'demos': required_demos_quarterly / weeks_per_quarter,
        'total_evaluations': required_evaluations_quarterly / weeks_per_quarter
    }
    
    # Break down evaluations by type
    for eval_type, percentage in eval_mix.items():
        eval_key = f"{eval_type.replace('_percentage', '')}_evaluations"
        weekly_activities[eval_key] = weekly_activities['total_evaluations'] * (percentage / 100)
    
    return weekly_activities


def calculate_evaluation_workload(weekly_evaluations_by_type, time_per_type):
    """
    Calculate SE time requirements for different evaluation types.
    
    Parameters:
    - weekly_evaluations_by_type (dict): Weekly evaluation counts by type
    - time_per_type (dict): Time requirements for each evaluation type
    
    Returns:
    - dict: Weekly hours needed by evaluation type
    """
    evaluation_hours = {}
    
    # Self-guided: ongoing weekly support hours
    if 'self_guided' in weekly_evaluations_by_type:
        evaluation_hours['self_guided'] = (weekly_evaluations_by_type['self_guided'] * 
                                         time_per_type['self_guided_support_hours_per_week'])
    
    # SE-led: ongoing weekly support hours  
    if 'se_led' in weekly_evaluations_by_type:
        evaluation_hours['se_led'] = (weekly_evaluations_by_type['se_led'] * 
                                    time_per_type['se_led_eval_hours_per_week'])
    
    # Rapid POV: total hours distributed over evaluation period (assume 2 weeks)
    if 'rapid_pov' in weekly_evaluations_by_type:
        rapid_pov_weeks = 2  # Standard duration for rapid POV
        evaluation_hours['rapid_pov'] = (weekly_evaluations_by_type['rapid_pov'] * 
                                       time_per_type['rapid_pov_total_hours'] / rapid_pov_weeks)
    
    # No evaluation type requires no SE time
    evaluation_hours['no_eval'] = 0
    
    return evaluation_hours


def calculate_demo_workload(weekly_demos, demo_time_params):
    """
    Calculate total demo workload including prep, delivery, and follow-up.
    
    Parameters:
    - weekly_demos (float): Number of demos per week
    - demo_time_params (dict): Time parameters for demo activities
    
    Returns:
    - float: Total weekly hours for demo activities
    """
    total_demo_time_per_demo = (demo_time_params['demo_prep_time'] + 
                               demo_time_params['demo_delivery_time'] + 
                               demo_time_params['demo_followup_time'])
    
    return weekly_demos * total_demo_time_per_demo


def calculate_meeting_workload(weekly_meetings, meeting_time):
    """
    Calculate workload for initial meetings including prep time.
    
    Parameters:
    - weekly_meetings (float): Number of initial meetings per week
    - meeting_time (float): Time per meeting including prep
    
    Returns:
    - float: Total weekly hours for initial meetings
    """
    return weekly_meetings * meeting_time


def calculate_account_meetings(account_counts, meeting_frequencies):
    """
    Calculate total meetings per month for existing accounts.
    
    Parameters:
    - account_counts (dict): Number of accounts by segment
    - meeting_frequencies (dict): Meeting frequency by segment (meetings/month)
    
    Returns:
    - dict: Monthly meetings by account segment
    """
    monthly_meetings = {}
    
    for segment, count in account_counts.items():
        if segment in meeting_frequencies:
            if segment == 'retain':
                # Retain accounts: frequency is "every X months", so meetings/month = 1/X
                meetings_per_month = count / meeting_frequencies[segment]
            else:
                # Other segments: frequency is meetings per month
                meetings_per_month = count * meeting_frequencies[segment]
            
            monthly_meetings[segment] = meetings_per_month
    
    return monthly_meetings


def calculate_new_logo_onboarding_meetings(new_logos_per_quarter, onboarding_params):
    """
    Calculate new customer onboarding meeting requirements.
    
    Parameters:
    - new_logos_per_quarter (float): New customers per quarter
    - onboarding_params (dict): Onboarding meeting parameters
    
    Returns:
    - dict: Monthly onboarding meeting requirements
    """
    monthly_onboarding = {}
    
    # Monthly meetings: 50% of new logos need 6 months of meetings
    monthly_customers = (new_logos_per_quarter * 
                        (onboarding_params['monthly_onboarding_percentage'] / 100))
    # 6 months of meetings spread across year = 6/12 = 0.5 factor
    monthly_onboarding['monthly_meetings'] = monthly_customers * 0.5
    
    # Quarterly meetings: 50% of new logos need 4 meetings per year
    quarterly_customers = (new_logos_per_quarter * 
                          (onboarding_params['quarterly_onboarding_percentage'] / 100))
    # 4 meetings per year = 4/12 meetings per month
    monthly_onboarding['quarterly_meetings'] = quarterly_customers * (4/12)
    
    return monthly_onboarding


def calculate_account_mgmt_hours(monthly_meetings, meeting_time_params):
    """
    Convert monthly meetings to weekly SE hours.
    
    Parameters:
    - monthly_meetings (dict): Monthly meeting counts by type
    - meeting_time_params (dict): Time per meeting including follow-up
    
    Returns:
    - float: Weekly hours for account management
    """
    total_monthly_meetings = sum(monthly_meetings.values())
    
    time_per_meeting = (meeting_time_params['meeting_duration_hours'] + 
                       meeting_time_params['meeting_followup_hours'])
    
    monthly_hours = total_monthly_meetings * time_per_meeting
    
    # Convert to weekly hours (assume 4.33 weeks per month)
    weekly_hours = monthly_hours / 4.33
    
    return weekly_hours


def calculate_strategic_workload(strategic_params, staffing_config):
    """
    Calculate strategic activity hours per role.
    
    Parameters:
    - strategic_params (dict): Strategic activity time parameters
    - staffing_config (dict): Team structure configuration
    
    Returns:
    - dict: Strategic hours by role type
    """
    # Calculate total strategic hours per week
    total_strategic_hours = 0
    
    # Handle min/max ranges by taking midpoint
    customer_zero_hours = (strategic_params['customer_zero_hours_min'] + 
                          strategic_params['customer_zero_hours_max']) / 2
    
    developer_advocacy_hours = (strategic_params['developer_advocacy_hours_min'] + 
                               strategic_params['developer_advocacy_hours_max']) / 2
    
    total_strategic_hours = (customer_zero_hours + 
                           strategic_params['content_creation_hours'] +
                           strategic_params['sales_enablement_hours'] +
                           developer_advocacy_hours +
                           strategic_params['asset_development_hours_avg'])
    
    return distribute_strategic_work(total_strategic_hours, 
                                   staffing_config['ic_strategic_percentage'],
                                   staffing_config['num_ic_ses'],
                                   staffing_config['num_directors'],
                                   staffing_config['director_ic_percentage'])


def distribute_strategic_work(total_hours, ic_percentage, num_ics, num_directors, director_ic_percentage):
    """
    Distribute strategic work between ICs and Directors.
    
    Parameters:
    - total_hours (float): Total strategic hours per week
    - ic_percentage (float): Percentage allocated to ICs
    - num_ics (int): Number of IC SEs
    - num_directors (int): Number of directors
    - director_ic_percentage (float): Percentage of director time on IC activities
    
    Returns:
    - dict: Strategic hours by role
    """
    ic_strategic_hours = total_hours * (ic_percentage / 100)
    director_strategic_hours = total_hours * ((100 - ic_percentage) / 100)
    
    # Calculate effective director capacity for strategic work
    # Directors spend (100 - director_ic_percentage)% on strategic activities
    director_strategic_capacity_percentage = 100 - director_ic_percentage
    
    return {
        'ic_strategic_hours_per_se': ic_strategic_hours / max(num_ics, 1),
        'director_strategic_hours_per_director': director_strategic_hours / max(num_directors, 1),
        'total_ic_strategic_hours': ic_strategic_hours,
        'total_director_strategic_hours': director_strategic_hours,
        'director_strategic_capacity_percentage': director_strategic_capacity_percentage
    }


def validate_percentages_sum_to_100(percentage_dict, tolerance=0.01):
    """
    Validate that percentages sum to 100%.
    
    Parameters:
    - percentage_dict (dict): Dictionary of percentage values
    - tolerance (float): Allowed deviation from 100%
    
    Returns:
    - bool: True if valid, False otherwise
    """
    total = sum(percentage_dict.values())
    return abs(total - 100) <= tolerance


def calculate_weekly_capacity(num_ses, hours_per_week=40):
    """
    Calculate total weekly capacity for SE team.
    
    Parameters:
    - num_ses (float): Number of SEs (can be fractional for part-time)
    - hours_per_week (float): Standard hours per week per SE
    
    Returns:
    - float: Total weekly capacity in hours
    """
    return num_ses * hours_per_week


def format_hours_for_display(hours):
    """
    Format hours for user-friendly display.
    
    Parameters:
    - hours (float): Hours to format
    
    Returns:
    - str: Formatted string
    """
    if hours < 0.1:
        return "< 0.1 hrs"
    elif hours < 1:
        return f"{hours:.1f} hrs"
    else:
        return f"{hours:.1f} hrs"


# Test helper functions with sample data
def test_helper_functions():
    """Test all helper functions with sample inputs to verify correctness."""
    print("🧪 Testing Helper Functions")
    print("=" * 40)
    
    test_results = []
    
    try:
        # Test revenue calculation
        result = calculate_required_new_logos(2000000, 75000)
        expected = 26.67
        assert abs(result - expected) < 0.1, f"Expected ~{expected}, got {result}"
        test_results.append("✅ calculate_required_new_logos: Working correctly")
        
        # Test win rate calculation
        eval_mix = {'self_guided_percentage': 65, 'se_led_percentage': 35}
        win_rates = {'self_guided_percentage': 35, 'se_led_percentage': 45}
        result = calculate_weighted_win_rate(eval_mix, win_rates)
        expected = 0.385  # (0.65 * 0.35) + (0.35 * 0.45)
        assert abs(result - expected) < 0.01, f"Expected ~{expected}, got {result}"
        test_results.append("✅ calculate_weighted_win_rate: Working correctly")
        
        # Test capacity calculation
        result = calculate_weekly_capacity(2.5, 40)
        expected = 100
        assert result == expected, f"Expected {expected}, got {result}"
        test_results.append("✅ calculate_weekly_capacity: Working correctly")
        
        # Test percentage validation
        valid_percentages = {'a': 30, 'b': 70}
        invalid_percentages = {'a': 30, 'b': 80}
        assert validate_percentages_sum_to_100(valid_percentages) == True
        assert validate_percentages_sum_to_100(invalid_percentages) == False
        test_results.append("✅ validate_percentages_sum_to_100: Working correctly")
        
        # Test hours formatting
        result = format_hours_for_display(12.567)
        expected = "12.6 hrs"
        assert result == expected, f"Expected {expected}, got {result}"
        test_results.append("✅ format_hours_for_display: Working correctly")
        
    except Exception as e:
        test_results.append(f"❌ Helper function test failed: {str(e)}")
    
    # Display results
    for result in test_results:
        print(result)
    
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print(f"\n📊 Test Summary: {success_count}/{total_count} tests passed")
    
    if success_count == total_count:
        print("🎉 All helper function tests passed!")
        return True
    else:
        print("⚠️  Some helper function tests failed.")
        return False

# Run tests
test_helper_functions()

print("\n✅ Helper Functions Framework implemented successfully")
print("📝 Ready for Step 3: Parameter Management")

---

# 2. Parameter Management {#parameter-management}

*Input parameter definitions, default values, and validation rules*

In [ ]:
# Placeholder for parameter management
# Will be implemented in Step 3

print("⚙️ Parameter management section ready for implementation")

---

# 3. Revenue & Pipeline Calculations {#revenue-pipeline}

*Core funnel math and workload calculations driven by revenue targets*

In [ ]:
# Placeholder for revenue and pipeline calculations
# Will be implemented in Step 4

print("💰 Revenue & pipeline calculations section ready for implementation")

---

# 4. Account Management Logic {#account-management}

*Meeting frequency calculations and time allocation for existing customers*

In [ ]:
# Placeholder for account management logic
# Will be implemented in Step 5

print("👥 Account management logic section ready for implementation")

---

# 5. Strategic Activities Logic {#strategic-activities}

*Strategic time allocation and distribution between ICs and Directors*

In [ ]:
# Placeholder for strategic activities logic
# Will be implemented in Step 6

print("🎯 Strategic activities logic section ready for implementation")

---

# 6. Interactive Parameter Controls {#interactive-controls}

*Widget-based interface for parameter input and real-time updates*

In [ ]:
# Placeholder for interactive controls
# Will be implemented in Steps 7-9

print("🎛️ Interactive controls section ready for implementation")

---

# 7. Real-time Calculations {#real-time-calculations}

*Live calculation updates based on parameter changes*

In [ ]:
# Placeholder for real-time calculations
# Will be implemented in Step 9

print("⚡ Real-time calculations section ready for implementation")

---

# 8. Core Visualizations {#core-visualizations}

*Essential charts for utilization analysis and capacity planning*

In [ ]:
# Placeholder for core visualizations
# Will be implemented in Step 10

print("📊 Core visualizations section ready for implementation")

---

# 9. Executive Dashboard {#executive-dashboard}

*Professional dashboard with comprehensive business insights*

In [ ]:
# Placeholder for executive dashboard
# Will be implemented in Step 11

print("📈 Executive dashboard section ready for implementation")

---

# 10. Results Summary & Export {#results-summary}

*Executive summary generation and export capabilities*

In [ ]:
# Placeholder for results summary and export
# Will be implemented in Step 12

print("📋 Results summary & export section ready for implementation")

---

# 11. Testing Framework {#testing}

*Comprehensive validation and testing of all calculations*

In [ ]:
# Placeholder for testing framework
# Will be implemented throughout all steps

print("🧪 Testing framework section ready for implementation")

---

# 12. Usage Examples {#examples}

*Sample scenarios and practical use cases*

In [ ]:
# Placeholder for usage examples
# Will be implemented in final step

print("💡 Usage examples section ready for implementation")

---

## Import Verification Test

*Basic test to verify all imports are working correctly*

In [ ]:
# Test all imported libraries
def test_imports():
    """Verify all required libraries are properly imported and functional."""
    test_results = []
    
    # Test pandas
    try:
        test_df = pd.DataFrame({'test': [1, 2, 3]})
        assert len(test_df) == 3
        test_results.append("✅ pandas: Working correctly")
    except Exception as e:
        test_results.append(f"❌ pandas: Error - {str(e)}")
    
    # Test numpy
    try:
        test_array = np.array([1, 2, 3])
        assert np.sum(test_array) == 6
        test_results.append("✅ numpy: Working correctly")
    except Exception as e:
        test_results.append(f"❌ numpy: Error - {str(e)}")
    
    # Test matplotlib
    try:
        fig, ax = plt.subplots(1, 1, figsize=(4, 3))
        ax.plot([1, 2, 3], [1, 4, 2])
        plt.close(fig)
        test_results.append("✅ matplotlib: Working correctly")
    except Exception as e:
        test_results.append(f"❌ matplotlib: Error - {str(e)}")
    
    # Test seaborn
    try:
        current_style = sns.axes_style()
        assert isinstance(current_style, dict)
        test_results.append("✅ seaborn: Working correctly")
    except Exception as e:
        test_results.append(f"❌ seaborn: Error - {str(e)}")
    
    # Test ipywidgets
    try:
        test_widget = widgets.IntSlider(value=5, min=0, max=10)
        assert test_widget.value == 5
        test_results.append("✅ ipywidgets: Working correctly")
    except Exception as e:
        test_results.append(f"❌ ipywidgets: Error - {str(e)}")
    
    # Display results
    print("\n🔍 Import Verification Results:")
    print("=" * 40)
    for result in test_results:
        print(result)
    
    # Summary
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print("\n📊 Summary:")
    print(f"   • {success_count}/{total_count} libraries working correctly")
    
    if success_count == total_count:
        print("\n🎉 All imports successful! Ready to proceed with implementation.")
        return True
    else:
        print("\n⚠️  Some imports failed. Please check your environment.")
        return False

# Run the test
test_imports()

---

## Next Steps

✅ **Step 1 Complete**: Basic notebook structure created with all required sections

### Ready for Implementation:
- **Step 2**: Helper Functions Framework - Core calculation functions
- **Step 3**: Parameter Management - Input parameter definitions and validation

### Implementation Notes:
- All libraries imported and tested successfully
- Professional styling and configuration applied
- Clear section structure with table of contents navigation
- Placeholder cells ready for modular development
- Testing framework initialized

**📝 Continue with Step 2 to implement the helper functions framework.**